In [1]:
import sys,re,random
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import nest_asyncio
import aiohttp
import asyncio
import requests

In [2]:
nest_asyncio.apply()

# URL

In [3]:
async def fetch(session, url):
    async with session.get(url) as html:
        return await html.text()

In [4]:
async def parser(html):
    soup=bs(html,'lxml')
    #time.sleep(random.uniform(0.5,1.5))
    urls_=soup.select('h3[class="width465px"]>a')
    sl=soup.select('span[class="recop-base-income"]>span[class="wage"]')
    for i in range(len(urls_)):
        #月額除外
        if len(re.findall(r"\d*\,*\d+",str(sl[i]))[0])>=3:
            urls.append("https://crew.lawson.co.jp"+re.findall(r"<a href=\"(.+)\"><",str(urls_[i]))[0])

In [5]:
async def download(url):
    async with aiohttp.ClientSession() as session:
        html = await fetch(session, url) 
        await parser(html)

In [ ]:
urls=[]
main_url="https://crew.lawson.co.jp/jobfind-pc/area/HokkaidoTohoku/All?jobtype=00001,00002,00003,00004,00005,00006,00008,00009,00010,00012&page="

#ページ数とる
html=requests.get(main_url)
soup=bs(html.text,'lxml')
page_sum=soup.select('span>strong')
page_=int(re.findall(r"\d+",str(page_sum))[0])
page=round(page_/20)+1

#非同期ループ
loop = asyncio.get_event_loop()
tasks = [asyncio.ensure_future(download(main_url+str(i))) for i in range(1,page)]
tasks = asyncio.gather(*tasks)
loop.run_until_complete(tasks)

urls=set(urls)

# Store name, address, hourly wage

In [30]:
async def parser_ads(html):
    soup=bs(html,'lxml')
    #time.sleep(random.uniform(0.5,1.5))
    #店舗名
    name_=soup.select('div[class="title"] > h2')
    if name_:
        name.append(re.findall(r">(.+)<",str(name_))[0])
        #住所
        address=soup.select('td>label')
        ads.append(re.findall(r">(.+)<",str(address))[0])
        #時給
        sl=str(soup.select('div[class="recop-base-income"] > span[class="wage"]')).replace(",","")
        salary.append(int(re.findall(r"\d+",sl)[0]))

async def download_ads(url,semaphore):
    async with semaphore:
        async with aiohttp.ClientSession() as session:
            html = await fetch(session, url)
            await parser_ads(html)

In [31]:
async def download_ads(url):
    async with aiohttp.ClientSession() as session:
        html = await fetch(session, url)
        await parser_ads(html)

In [ ]:
ads=[]
salary=[]
name=[]
loop = asyncio.get_event_loop()
#semaphore = asyncio.Semaphore(500)
tasks = [asyncio.ensure_future(download_ads(url)) for url in urls]
tasks = asyncio.gather(*tasks)
loop.run_until_complete(tasks)

In [36]:
df=pd.read_pickle(r"ce.pkl")

In [37]:
shop=["ローソン" for i in range(len(name))]
shop1=pd.DataFrame(shop)
name1=pd.DataFrame(name)
ads1=pd.DataFrame(ads)
salary1=pd.DataFrame(salary)

In [38]:
cc=pd.concat([shop1,name1,ads1,salary1],axis=1)
cc.columns=["shop","name","address","salary"]
cc=pd.concat([df,cc],axis=0)
cc.to_pickle(r"ce.pkl")

In [47]:
len(urls)

1404

In [39]:
cc.to_csv(r"convenience_store.csv",encoding="utf_8_sig",index=False)

In [48]:
hokaido=pd.DataFrame({"shop":pd.Series(["セブンイレブン","ファミリーマート","ローソン"]),
             "time":pd.Series([58.391958236694336,103.62012648582458,50.73404240608215]),
                     "url_sum":pd.Series([796,199,1404])})

In [49]:
hokaido.to_csv(r"hokaido.csv",encoding="utf_8_sig",index=False)

# Store latitude and longitude, business hours

In [ ]:
lat=[]
lng=[]
tm=[]
main_url="https://www.google.com/maps/"
options = Options()
driver = webdriver.Chrome(options = options)
driver.get(main_url)
for i in tqdm(range(len(name))):
    #google map 検索-入力
    driver.find_element_by_id('searchboxinput').send_keys(name[i])
    driver.find_element_by_id('searchbox-searchbutton').click()
    time.sleep(3)
    lat_=re.findall(r"3d(\d+\.*\d+)!4d",driver.current_url)[0]
    lng_=re.findall(r".+4d(.+)$",driver.current_url)[0]
    lat.append(lat_)
    lng.append(lng_)
    
    tm.append(driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[12]/div[1]/span[2]/span[2]').text)
    
    driver.find_element_by_id('searchboxinput').clear()
driver.quit()

# Data storage

In [ ]:
name1=pd.DataFrame(name)
ads1=pd.DataFrame(ads)
tm1=pd.DataFrame(tm)
salary1==pd.DataFrame(salary)
lat1=pd.DataFrame(lat)
lng1=pd.DataFrame(lng)

In [ ]:
cc=pd.concat([name1,ads1,tm1,salary1,lat1,lng1],axis=1)
cc.columns=["name","address","business_hours","salary","lat","lng"]
cc.to_csv(r"lawson.csv",encoding="utf_8_sig",index=False)